<a href="https://colab.research.google.com/github/fundaylncii/RecommendationSystems/blob/main/MovieRecommendationWithItemBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## İtem Based

In [2]:
## Item Based Collaborative Filtering
## Item benzerlikleri üzerinden öneriler yapılır
## beğenilme yapısı benzerliği aranır

## DataSet Url: https://grouplens.org/datasets/movielens/

## Veri Setinin Hazırlanması
import pandas as pd
pd.set_option("display.max_columns", 20)

movie = pd.read_csv("/content/movie.csv")
rating = pd.read_csv("/content/rating.csv")

df = movie.merge(rating, how="left", on="movieId")

In [6]:
df.head(10)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12.0,4.0,1997-03-22 20:48:38
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,4.0,1996-11-27 08:21:31
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,2008-10-29 20:13:59
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,16.0,3.0,2001-05-27 13:22:52
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,5.0,1997-02-05 21:03:48


In [11]:
## User Movie Df Oluşturulması
## önem taşımayan az sayıda yorum alan filmler çıkartılarak analiz gerçekleştirilmelidir.

df.shape
df["title"].nunique()
df["title"].value_counts().head()


title
Pulp Fiction (1994)                 5312
Forrest Gump (1994)                 5297
Silence of the Lambs, The (1991)    4956
Shawshank Redemption, The (1994)    4913
Jurassic Park (1993)                4748
Name: count, dtype: int64

In [13]:
## analiz açısından 1000'den az yorum alan filmler çıkartılacaktır
comments_count = pd.DataFrame(df["title"].value_counts())
print(comments_count)


                                                    count
title                                                    
Pulp Fiction (1994)                                  5312
Forrest Gump (1994)                                  5297
Silence of the Lambs, The (1991)                     4956
Shawshank Redemption, The (1994)                     4913
Jurassic Park (1993)                                 4748
...                                                   ...
Stewart Lee: If You Prefer a Milder Comedian, P...      1
Look What's Happened to Rosemary's Baby (a.k.a....      1
Great Stone Face, The (1968)                            1
Speed & Angels (2008)                                   1
Innocence (2014)                                        1

[27262 rows x 1 columns]


In [16]:
## 1000 den az yorum alanlar:
rare_movie = comments_count[comments_count["count"] <= 1000].index
print(rare_movie)

Index(['City Slickers II: The Legend of Curly's Gold (1994)',
       'Notting Hill (1999)', 'Johnny Mnemonic (1995)', 'Casino Royale (2006)',
       'Father of the Bride Part II (1995)', 'Disclosure (1994)',
       'Thelma & Louise (1991)', 'Three Musketeers, The (1993)',
       'Fantasia (1940)', 'Bowling for Columbine (2002)',
       ...
       'Christmas Holiday (1944)', 'Back Street (1961)', 'Up in Arms (1944)',
       'Traveler, The (2010)', 'Chain Letter (2009)',
       'Stewart Lee: If You Prefer a Milder Comedian, Please Ask for One (2010)',
       'Look What's Happened to Rosemary's Baby (a.k.a. Rosemary's Baby II) (1976)',
       'Great Stone Face, The (1968)', 'Speed & Angels (2008)',
       'Innocence (2014)'],
      dtype='object', name='title', length=26923)


In [17]:
## df içerisinden 1000 den az yorum olmayanların oluşturulması
common_movies = df[~df["title"].isin(rare_movie)]
print(common_movies)

         movieId             title  \
0              1  Toy Story (1995)   
1              1  Toy Story (1995)   
2              1  Toy Story (1995)   
3              1  Toy Story (1995)   
4              1  Toy Story (1995)   
...          ...               ...   
1576574    79132  Inception (2010)   
1576575    79132  Inception (2010)   
1576576    79132  Inception (2010)   
1576577    79132  Inception (2010)   
1614198   114240    Aladdin (1992)   

                                                  genres   userId  rating  \
0            Adventure|Animation|Children|Comedy|Fantasy      3.0     4.0   
1            Adventure|Animation|Children|Comedy|Fantasy      6.0     5.0   
2            Adventure|Animation|Children|Comedy|Fantasy      8.0     4.0   
3            Adventure|Animation|Children|Comedy|Fantasy     10.0     4.0   
4            Adventure|Animation|Children|Comedy|Fantasy     11.0     4.5   
...                                                  ...      ...     ...   
1576

In [21]:
## Yüksek puanlı eşsiz film sayısı
common_movies["title"].nunique()

339

In [23]:
## user df creating
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
print(user_movie_df)

title    12 Angry Men (1957)  2001: A Space Odyssey (1968)  Abyss, The (1989)  \
userId                                                                          
1.0                      NaN                           3.5                NaN   
2.0                      NaN                           5.0                NaN   
3.0                      NaN                           5.0                3.0   
4.0                      NaN                           NaN                NaN   
5.0                      NaN                           NaN                NaN   
...                      ...                           ...                ...   
10825.0                  NaN                           NaN                NaN   
10826.0                  NaN                           5.0                4.0   
10827.0                  NaN                           NaN                NaN   
10828.0                  NaN                           NaN                NaN   
10829.0                  NaN

In [24]:
user_movie_df.shape

(10721, 339)

In [31]:
## Item Based Film Önerisi Oluşturma

pd.set_option("display.max_columns",500)
movie_name = "Matrix, The (1999)"
movie_name = user_movie_df[movie_name]
# print(movie_name)
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)


title
Matrix, The (1999)                                   1.000000
Matrix Reloaded, The (2003)                          0.501695
Terminator 2: Judgment Day (1991)                    0.340030
Inception (2010)                                     0.321155
Men in Black (a.k.a. MIB) (1997)                     0.320808
Lord of the Rings: The Two Towers, The (2002)        0.320497
Terminator, The (1984)                               0.319067
Top Gun (1986)                                       0.317431
Batman Begins (2005)                                 0.316988
Star Wars: Episode VI - Return of the Jedi (1983)    0.309763
dtype: float64

In [32]:
movie_name = "Terminator, The (1984)"
movie_name = user_movie_df[movie_name]
# print(movie_name)
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)


title
Terminator, The (1984)                                                            1.000000
Terminator 2: Judgment Day (1991)                                                 0.609855
Aliens (1986)                                                                     0.485436
RoboCop (1987)                                                                    0.463676
Die Hard (1988)                                                                   0.458359
Total Recall (1990)                                                               0.457005
Alien (1979)                                                                      0.441642
Predator (1987)                                                                   0.429403
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    0.426091
Lethal Weapon (1987)                                                              0.406933
dtype: float64

In [34]:
def check_film(keyboard, user_movie_df):
  return [col for col in user_movie_df.columns if keyboard in col]

check_film("Matrix", user_movie_df)

['Matrix Reloaded, The (2003)', 'Matrix, The (1999)']

In [35]:
def item_based_recommender(movie_name, user_movie_df):
  movie_name = user_movie_df[movie_name]
  return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [36]:
item_based_recommender("Terminator, The (1984)", user_movie_df)

title
Terminator, The (1984)                                                            1.000000
Terminator 2: Judgment Day (1991)                                                 0.609855
Aliens (1986)                                                                     0.485436
RoboCop (1987)                                                                    0.463676
Die Hard (1988)                                                                   0.458359
Total Recall (1990)                                                               0.457005
Alien (1979)                                                                      0.441642
Predator (1987)                                                                   0.429403
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    0.426091
Lethal Weapon (1987)                                                              0.406933
dtype: float64

In [38]:
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
print(movie_name)

Babe (1995)


In [39]:
item_based_recommender(movie_name, user_movie_df)

title
Babe (1995)                                       1.000000
Toy Story 2 (1999)                                0.470459
Toy Story (1995)                                  0.441664
Bug's Life, A (1998)                              0.439738
Finding Nemo (2003)                               0.400345
Wizard of Oz, The (1939)                          0.389284
Snow White and the Seven Dwarfs (1937)            0.387747
Wallace & Gromit: The Wrong Trousers (1993)       0.382463
Harry Potter and the Chamber of Secrets (2002)    0.379881
Monsters, Inc. (2001)                             0.373221
dtype: float64